In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('data/train.csv')
train

,index,userId,movieId,rating
0,10957246,113324,92,2.0
1,19270699,200168,2028,4.5
2,18196248,188774,185,4.0
3,18087240,187777,4932,5.0
4,727218,7405,1965,2.0
...,...,...,...,...
8710063,6676816,68964,26539,3.0
8710064,674492,6881,163,2.5
8710065,17771894,184449,89492,4.5
8710066,22559319,234296,56372,4.5


In [3]:
test = pd.read_csv('data/test.csv')
test

,index,userId,movieId,rating
0,126011,1258,3210,5.0
1,9384081,96937,161,5.0
2,22372226,232166,524,4.0
3,10556437,108970,3104,4.0
4,22206955,230482,1089,5.0
...,...,...,...,...
2177513,5301105,54695,150,1.5
2177514,3544450,36922,3418,4.0
2177515,2506343,26036,805,2.5
2177516,18409157,191069,3527,1.0


In [10]:
from sklearn.metrics import ndcg_score
from trivial_rankers import VoteAverageBasedRanker
from ipywidgets import IntProgress
from IPython.display import display

r = VoteAverageBasedRanker(pd.read_csv('data/movies.csv'), train)
test_users = test['userId'].unique()[:10000]
progress = IntProgress(min=0, max=len(test_users))
display(progress)

scores = []
for user in test_users:
    progress.value += 1

    user_test: np.ndarray = test[test['userId'] == user] \
        .sort_values('rating', ascending=False)['movieId'] \
        .to_numpy()
    
    rankings = r.rank(user, user_test)
    score = ndcg_score([user_test], [rankings], k=5)
    scores.append(score)

print(sum(scores) / len(scores))

IntProgress(value=0, max=10000)

0.4373015037138988
